In [1]:
import yaml
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'
import torch
import torch.nn as nn
import sys
sys.path.append('/data/ouyuan/yolov5')
from tqdm import tqdm
from utils.dataloaders import create_dataloader
from moe import MoE, ModelSuffix, ModelName
from pathlib import Path
import copy
import numpy as np
import random
import glob
from utils.general import (
    LOGGER,
    TQDM_BAR_FORMAT,
    Profile,
    check_amp,
    check_dataset,
    check_file,
    check_git_info,
    check_git_status,
    check_img_size,
    check_requirements,
    check_suffix,
    check_yaml,
    colorstr,
    get_latest_run,
    increment_path,
    init_seeds,
    intersect_dicts,
    labels_to_class_weights,
    labels_to_image_weights,
    methods,
    one_cycle,
    print_args,
    print_mutation,
    strip_optimizer,
    yaml_save,
    increment_path,
    non_max_suppression,
    scale_boxes,
    xywh2xyxy,
    xyxy2xywh,
)
from utils.torch_utils import (
    EarlyStopping,
    ModelEMA,
    de_parallel,
    select_device,
    smart_DDP,
    smart_optimizer,
    smart_resume,
    torch_distributed_zero_first,
)
from utils.loss import ComputeLoss
from utils.metrics import ConfusionMatrix, ap_per_class, box_iou
from utils.plots import output_to_target, plot_images, plot_val_study

In [2]:
LOCAL_RANK = int(os.getenv("LOCAL_RANK", -1))
RANK = int(os.getenv("RANK", -1))
WORLD_SIZE = int(os.getenv("WORLD_SIZE", 1))
torch.backends.cudnn.enabled = False
data = 'cityscapes.yaml'
imgsz = 640
batch_size = 32
gs = 32
workers = 8
label_smoothing = 0.0
seed = 19260817
init_seeds(seed + 1 + RANK, deterministic=True)
single_cls = False
epochs = 100
optimizer = 'Adam'
nc = 80
data = check_yaml(data)
hyp = '../data/hyps/hyp.scratch-low.yaml'
with open(hyp, errors="ignore") as f:
    hyp = yaml.safe_load(f)
data_dict = check_dataset(data)
train_path, val_path = data_dict["train"], data_dict["val"]
train_loader, dataset = create_dataloader(
        train_path,
        imgsz,
        batch_size,
        gs,
        single_cls,
        hyp=hyp,
        rank=LOCAL_RANK,
        prefix=colorstr("train: "),
        shuffle=True,
        seed=seed,
    )

moe = MoE()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
nl = de_parallel(moe.experts[0]).model[-1].nl  # number of detection layers (to scale hyps)
hyp["box"] *= 3 / nl  # scale to layers
hyp["cls"] *= nc / 80 * 3 / nl  # scale to classes and layers
hyp["obj"] *= (imgsz / 640) ** 2 * 3 / nl  # scale to image size and layers
hyp["label_smoothing"] = label_smoothing

moe.to(device)

compute_loss1 = ComputeLoss(moe.experts[0])
compute_loss2 = ComputeLoss(moe.experts[5])

train: Scanning /data/ouyuan/datasets/cityscapes/train.cache... 2975 images, 7 backgrounds, 0 corrupt: 100%|██████████| 2975/2975 [00:00<?, ?it/s]
train: WARNING ⚠️ /data/ouyuan/datasets/cityscapes/images/train/aachen/aachen_000000_000019_leftImg8bit.png: 3 duplicate labels removed
train: WARNING ⚠️ /data/ouyuan/datasets/cityscapes/images/train/aachen/aachen_000022_000019_leftImg8bit.png: 1 duplicate labels removed
train: WARNING ⚠️ /data/ouyuan/datasets/cityscapes/images/train/aachen/aachen_000030_000019_leftImg8bit.png: 3 duplicate labels removed
train: WARNING ⚠️ /data/ouyuan/datasets/cityscapes/images/train/aachen/aachen_000031_000019_leftImg8bit.png: 1 duplicate labels removed
train: WARNING ⚠️ /data/ouyuan/datasets/cityscapes/images/train/aachen/aachen_000042_000019_leftImg8bit.png: 1 duplicate labels removed
train: WARNING ⚠️ /data/ouyuan/datasets/cityscapes/images/train/aachen/aachen_000054_000019_leftImg8bit.png: 1 duplicate labels removed
train: WARNING ⚠️ /data/ouyuan/datase

In [3]:
""" for i in range(moe.experts_num):
    for name, param in moe.experts[i].named_parameters():
        param.requires_grad = False """

# layer 0 - 9 for 640; 0 - 11 for 1280

for i in range(10):
    freeze = 10 if i < 5 else 12
    freeze = [f'model.{x}.' for x in range(freeze)]  # layers to freeze
    for k, v in moe.experts[i].named_parameters():
        v.requires_grad = True  # train all layers
        if any(x in k for x in freeze):
            #print(f'freezing {k}')
            v.requires_grad = False

for name, param in moe.latency_predictor.named_parameters():
        param.requires_grad = False
        
# for name, param in moe.named_parameters():
#     print(name)

In [4]:
# Optimizer
optimizer = 'Adam'
optimizer = smart_optimizer(moe, optimizer, hyp["lr0"], hyp["momentum"], hyp["weight_decay"])
#optimizer.add_param_group({'params': moe.w_gate, 'weight_decay': hyp["weight_decay"]})
# optimizer = torch.optim.Adam([{'params': moe.w_gate}], lr=hyp["lr0"], betas=(hyp["momentum"], 0.999))

nb = len(train_loader)
#amp = check_amp(moe)

#scaler = torch.cuda.amp.GradScaler(enabled=amp)

optimizer: Adam(lr=0.001) with parameter groups 964 weight(decay=0.0), 1006 weight(decay=0.0005), 1006 bias


In [5]:
save_dir = './exp_tmp'
conf_thres = 0.001
iou_thres = 0.6
max_det = 300
iouv = torch.linspace(0.5, 0.95, 10, device=device)  # iou vector for mAP@0.5:0.95
niou = iouv.numel()
seen = 0
plots = True
save_dir = Path(save_dir)
confusion_matrix = ConfusionMatrix(nc=nc)
names = moe.experts[0].names if hasattr(moe.experts[0], "names") else moe.experts[0].module.names  # get class names
if isinstance(names, (list, tuple)):  # old format
    names = dict(enumerate(names))
task = 'val'
verbose = False
training = False

dt = Profile(device=device), Profile(device=device), Profile(device=device)  # profiling times

In [6]:
teacher = torch.load(os.path.join('..', 'yolov5x.pt'), map_location="cpu")["model"].float()
teacher.to(device)
teacher.eval()
teacher_conf_thres = 0.15
teacher_iou_thres = 0.6
for batch_i, (imgs, targets, paths, _) in enumerate(train_loader):
    imgs = imgs.to(device, non_blocking=True).float() / 255
    bs, _, height, width = imgs.shape
    preds, train_out = teacher(imgs)
    preds = preds.detach()
    # targets[:, 2:] *= torch.tensor((width, height, width, height), device=device)  # to pixels
    lb = [] # for autolabelling
    preds = non_max_suppression(
        preds, teacher_conf_thres, teacher_iou_thres, labels=lb, multi_label=True, agnostic=single_cls, max_det=max_det
    )
    preds = torch.Tensor(output_to_target(preds)).cuda()
    preds = preds[:,:-1]
    preds[:, 2:] /= torch.tensor((width, height, width, height), device=device)
    print(preds.shape)
    print(targets.device)
    print(preds[0])    
    print(targets[0])
    break
    

torch.Size([660, 6])
cpu
tensor([0.00000, 2.00000, 0.22496, 0.51735, 0.29484, 0.16851], device='cuda:0')
tensor([0.00000, 2.00000, 0.39160, 0.45801, 0.01074, 0.00684])


In [7]:
experts_moe = []
experts_moe_id = []
pbar = enumerate(train_loader)
teacher.eval()
LOGGER.info(("\n" + "%11s" * 7) % ("Epoch", "GPU_mem", "box_loss", "obj_loss", "cls_loss", "expert_id", "Size"))
if RANK in {-1, 0}:
    pbar = tqdm(pbar, total=nb, bar_format=TQDM_BAR_FORMAT)  # progress bar
for batch_i, (imgs, targets, paths, _) in pbar:  # window -------------------------------------------------------------
    if batch_i == 0:
        id = random.randint(0, 9)
        experts_moe_id.append(id)
        experts_moe.append(copy.deepcopy(moe.experts[id]).cpu())
    if batch_i == 10:
        break
    if batch_i == len(train_loader) - 1:
        continue
    torch.cuda.empty_cache()
    imgs = imgs.to(device, non_blocking=True).float() / 255  # uint8 to float32, 0-255 to 0.0-1.0
    bs, _, height, width = imgs.shape
    labels, train_out = teacher(imgs)
    labels = labels.detach()
    # targets[:, 2:] *= torch.tensor((width, height, width, height), device=device)  # to pixels
    lb = [] # for autolabelling
    labels = non_max_suppression(
        labels, teacher_conf_thres, teacher_iou_thres, labels=lb, multi_label=True, agnostic=single_cls, max_det=max_det
    )
    labels = torch.Tensor(output_to_target(labels)).cuda()
    labels = labels[:,:-1]
    labels[:, 2:] /= torch.tensor((width, height, width, height), device=device)
    # targets = targets.to(device)
    
    optimizer.zero_grad()
    for epoch in range(epochs):  # retrain ------------------------------------------------------------------
        moe.train()

        # Update mosaic border (optional)
        # b = int(random.uniform(0.25 * imgsz, 0.75 * imgsz + gs) // gs * gs)
        # dataset.mosaic_border = [b - imgsz, -b]  # height, width borders

        mloss = torch.zeros(3, device=device)  # mean losses
        # if RANK != -1:
        #     train_loader.sampler.set_epoch(epoch)
        
        # Forward
        #with torch.cuda.amp.autocast(amp):
        pred, id, latency = moe(imgs)  # forward
        # print(len(pred))
        if id < 5:
            loss, loss_items = compute_loss1(pred, labels)  # loss scaled by batch_size
        else:
            loss, loss_items = compute_loss2(pred, labels)  # loss scaled by batch_size
            
        if RANK != -1:
            loss *= WORLD_SIZE  # gradient averaged between devices in DDP mode
        
        if torch.any(torch.isnan(loss_items)):
            torch.cuda.empty_cache()
            bs, _, height, width = imgs.shape
            moe.experts[id].eval()
            preds, train_out = moe.experts[id](imgs)
            preds = preds.detach()
            targets[:, 2:] *= torch.tensor((width, height, width, height), device=device)  # to pixels
            lb = [] # for autolabelling
            preds = non_max_suppression(
                preds, conf_thres, iou_thres, labels=lb, multi_label=True, agnostic=single_cls, max_det=max_det
            )
            plot_images(imgs, targets, paths, save_dir / f"val_batch{batch_i}_id{id}_labels.jpg", names)  # labels
            plot_images(imgs, output_to_target(preds), paths, save_dir / f"val_batch{batch_i}_id{id}_pred.jpg", names)  # pred
            assert False
        
        #print(loss, latency)
        loss = loss + latency
        # Backward
        #scaler.scale(loss).backward()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Log
        if RANK in {-1, 0}:
            mloss = (mloss * i + loss_items) / (i + 1)  # update mean losses
            mem = f"{torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0:.3g}G"  # (GB)
            pbar.set_description(
                ("%11s" * 2 + "%11.4g" * 5)
                % (f"{epoch}/{epochs - 1}", mem, *mloss, id, imgs.shape[-1])
            )
        # end retrain ------------------------------------------------------------------------------------------------
        
    moe.eval()
    id = moe(imgs, switch=True)
    experts_moe_id.append(id.cpu().detach())
    experts_moe.append(copy.deepcopy(moe.experts[id]).cpu())
    imgs, labels = imgs.cpu(), labels.cpu()

    # end window ----------------------------------------------------------------------------------------------------
# end training -----------------------------------------------------------------------------------------------------


      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  expert_id       Size
      99/99      12.5G   0.001412   0.002376  0.0006589          7        640:  11%|█         | 10/93 [11:49<1:38:06, 70.92s/it]


In [8]:
print(len(experts_moe))
moe.to('cpu')
torch.cuda.empty_cache()
save_tmp = './exp_tmp'
save_dir = './exp_1'
cnt = 0
while(1):
    cnt += 1
    save_dir = save_tmp.replace('tmp', f'{cnt}')
    # print(f'Testing {save_dir}')
    if not os.path.exists(save_dir):
        print(f'Saving experts to {save_dir}')
        os.makedirs(save_dir)
        break

expert_dir = os.path.join(save_dir, 'moe')
for i, expert in enumerate(experts_moe):
    if not os.path.exists(expert_dir):
        os.makedirs(expert_dir)
    torch.save(expert, os.path.join(expert_dir, 'expert_%d_id_%d.ckpt'%(i, experts_moe_id[i])))

11
Saving experts to ./exp_2


In [9]:
gate = torch.zeros((16, 8)).sum(dim=0)
gate.shape

torch.Size([8])

In [10]:
def process_batch(detections, labels, iouv):
    """
    Return correct prediction matrix.

    Arguments:
        detections (array[N, 6]), x1, y1, x2, y2, conf, class
        labels (array[M, 5]), class, x1, y1, x2, y2
    Returns:
        correct (array[N, 10]), for 10 IoU levels
    """
    correct = np.zeros((detections.shape[0], iouv.shape[0])).astype(bool)
    iou = box_iou(labels[:, 1:], detections[:, :4])
    correct_class = labels[:, 0:1] == detections[:, 5]
    for i in range(len(iouv)):
        x = torch.where((iou >= iouv[i]) & correct_class)  # IoU > threshold and classes match
        if x[0].shape[0]:
            matches = torch.cat((torch.stack(x, 1), iou[x[0], x[1]][:, None]), 1).cpu().numpy()  # [label, detect, iou]
            if x[0].shape[0] > 1:
                matches = matches[matches[:, 2].argsort()[::-1]]
                matches = matches[np.unique(matches[:, 1], return_index=True)[1]]
                # matches = matches[matches[:, 2].argsort()[::-1]]
                matches = matches[np.unique(matches[:, 0], return_index=True)[1]]
            correct[matches[:, 1].astype(int), i] = True
    return torch.tensor(correct, dtype=torch.bool, device=iouv.device)

In [11]:
""" jdict, stats, ap, ap_class = [], [], [], []
save_dir = Path(save_dir)
loss = torch.zeros(3, device=device)
pbar = enumerate(train_loader)
LOGGER.info(("\n" + "%11s" * 7) % ("Epoch", "GPU_mem", "box_loss", "obj_loss", "cls_loss", "Instances", "Size"))
if RANK in {-1, 0}:
    pbar = tqdm(pbar, total=nb, bar_format=TQDM_BAR_FORMAT)  # progress bar
    
for batch_i, (im, targets, paths, shapes) in pbar:
    experts_moe[batch_i].to(device)
    experts_moe[batch_i].eval()
    with dt[0]:
        im = im.to(device, non_blocking=True)
        targets = targets.to(device)
        im = im.float()  # uint8 to fp16/32
        im /= 255  # 0 - 255 to 0.0 - 1.0
        bs, _, height, width = im.shape  # batch size, channels, height, width

    with dt[1]:
        preds, train_out = experts_moe[batch_i](im)
    preds = preds.detach()

    id = experts_moe_id[batch_i]
    # Loss
    if id < 5:
        loss += compute_loss1(train_out, targets)[1]  # box, obj, cls
    else:
        loss += compute_loss2(train_out, targets)[1]  # box, obj, cls

    # NMS
    targets[:, 2:] *= torch.tensor((width, height, width, height), device=device)  # to pixels
    lb = [] # for autolabelling

    with dt[2]:
        preds = non_max_suppression(
            preds, conf_thres, iou_thres, labels=lb, multi_label=True, agnostic=single_cls, max_det=max_det
        )

    # Metrics
    for si, pred in enumerate(preds):
        labels = targets[targets[:, 0] == si, 1:]
        nl, npr = labels.shape[0], pred.shape[0]  # number of labels, predictions
        path, shape = Path(paths[si]), shapes[si][0]
        correct = torch.zeros(npr, niou, dtype=torch.bool, device=device)  # init
        seen += 1

        if npr == 0:
            if nl:
                stats.append((correct, *torch.zeros((2, 0), device=device), labels[:, 0]))
                if plots:
                    confusion_matrix.process_batch(detections=None, labels=labels[:, 0])
            continue

        # Predictions
        if single_cls:
            pred[:, 5] = 0
        predn = pred.clone()
        scale_boxes(im[si].shape[1:], predn[:, :4], shape, shapes[si][1])  # native-space pred

        # Evaluate
        if nl:
            tbox = xywh2xyxy(labels[:, 1:5])  # target boxes
            scale_boxes(im[si].shape[1:], tbox, shape, shapes[si][1])  # native-space labels
            labelsn = torch.cat((labels[:, 0:1], tbox), 1)  # native-space labels
            correct = process_batch(predn, labelsn, iouv)
            if plots:
                confusion_matrix.process_batch(predn, labelsn)
        stats.append((correct, pred[:, 4], pred[:, 5], labels[:, 0]))  # (correct, conf, pcls, tcls)


    # Plot images
    if plots and batch_i < 3:
        plot_images(im, targets, paths, save_dir / f"val_batch{batch_i}_labels.jpg", names)  # labels
        plot_images(im, output_to_target(preds), paths, save_dir / f"val_batch{batch_i}_pred.jpg", names)  # pred
        
    experts_moe[batch_i].cpu()

# Compute metrics
stats = [torch.cat(x, 0).cpu().numpy() for x in zip(*stats)]  # to numpy
if len(stats) and stats[0].any():
    tp, fp, p, r, f1, ap, ap_class = ap_per_class(*stats, plot=plots, save_dir=save_dir, names=names)
    ap50, ap = ap[:, 0], ap.mean(1)  # AP@0.5, AP@0.5:0.95
    mp, mr, map50, map = p.mean(), r.mean(), ap50.mean(), ap.mean()
nt = np.bincount(stats[3].astype(int), minlength=nc)  # number of targets per class

# Print results
pf = "%22s" + "%11i" * 2 + "%11.3g" * 4  # print format
LOGGER.info(pf % ("all", seen, nt.sum(), mp, mr, map50, map))
if nt.sum() == 0:
    LOGGER.warning(f"WARNING ⚠️ no labels found in {task} set, can not compute metrics without labels")

# Print results per class
if (verbose or (nc < 50 and not training)) and nc > 1 and len(stats):
    for i, c in enumerate(ap_class):
        LOGGER.info(pf % (names[c], seen, nt[c], p[i], r[i], ap50[i], ap[i]))

# Print speeds
t = tuple(x.t / seen * 1e3 for x in dt)  # speeds per image
if not training:
    shape = (batch_size, 3, imgsz, imgsz)
    LOGGER.info(f"Speed: %.1fms pre-process, %.1fms inference, %.1fms NMS per image at shape {shape}" % t)

# Plots
if plots:
    confusion_matrix.plot(save_dir=save_dir, names=list(names.values()))

# Return results
if not training:
    s = ""
    LOGGER.info(f"Results saved to {colorstr('bold', save_dir)}{s}")
maps = np.zeros(nc) + map
for i, c in enumerate(ap_class):
    maps[c] = ap[i]
    
print((mp, mr, map50, map, *(loss.cpu() / len(train_loader)).tolist()), maps, t) """

' jdict, stats, ap, ap_class = [], [], [], []\nsave_dir = Path(save_dir)\nloss = torch.zeros(3, device=device)\npbar = enumerate(train_loader)\nLOGGER.info(("\n" + "%11s" * 7) % ("Epoch", "GPU_mem", "box_loss", "obj_loss", "cls_loss", "Instances", "Size"))\nif RANK in {-1, 0}:\n    pbar = tqdm(pbar, total=nb, bar_format=TQDM_BAR_FORMAT)  # progress bar\n    \nfor batch_i, (im, targets, paths, shapes) in pbar:\n    experts_moe[batch_i].to(device)\n    experts_moe[batch_i].eval()\n    with dt[0]:\n        im = im.to(device, non_blocking=True)\n        targets = targets.to(device)\n        im = im.float()  # uint8 to fp16/32\n        im /= 255  # 0 - 255 to 0.0 - 1.0\n        bs, _, height, width = im.shape  # batch size, channels, height, width\n\n    with dt[1]:\n        preds, train_out = experts_moe[batch_i](im)\n    preds = preds.detach()\n\n    id = experts_moe_id[batch_i]\n    # Loss\n    if id < 5:\n        loss += compute_loss1(train_out, targets)[1]  # box, obj, cls\n    else:\n

In [12]:
model_name = glob.glob(os.path.join(expert_dir, f'expert_{0}_id_*.ckpt'))[0]
id = int(model_name.split('/')[-1].split('.')[0].split('_')[-1])
# model = torch.load(os.path.join('..', 'yolov5x' + '.pt'), map_location="cpu")["model"].float()
# print(model.__dict__)

In [13]:
jdict, stats, ap, ap_class = [], [], [], []
save_dir = Path(save_dir)
loss = torch.zeros(3, device=device)
pbar = enumerate(train_loader)
LOGGER.info(("\n" + "%11s" * 7) % ("Epoch", "GPU_mem", "box_loss", "obj_loss", "cls_loss", "Instances", "Size"))
if RANK in {-1, 0}:
    pbar = tqdm(pbar, total=nb, bar_format=TQDM_BAR_FORMAT)  # progress bar
    
for batch_i, (im, targets, paths, shapes) in pbar:
    torch.cuda.empty_cache()
    if batch_i == 10:
        break
    model_name = glob.glob(os.path.join(expert_dir, f'expert_{batch_i}_id_*.ckpt'))[0]
    model = torch.load(model_name, map_location="cpu").float()
    model.to(device)
    model.eval()
    
    with dt[0]:
        im = im.to(device, non_blocking=True)
        targets = targets.to(device)
        im = im.float()  # uint8 to fp16/32
        im /= 255  # 0 - 255 to 0.0 - 1.0
        bs, _, height, width = im.shape  # batch size, channels, height, width

    
    teacher_labels, train_out = teacher(im)
    teacher_labels = teacher_labels.detach()
    # targets[:, 2:] *= torch.tensor((width, height, width, height), device=device)  # to pixels
    lb = [] # for autolabelling
    teacher_labels = non_max_suppression(
        teacher_labels, teacher_conf_thres, teacher_iou_thres, labels=lb, multi_label=True, agnostic=single_cls, max_det=max_det
    )
    teacher_labels = torch.Tensor(output_to_target(teacher_labels)).cuda()
    teacher_labels = teacher_labels[:,:-1]
    teacher_labels[:, 2:] /= torch.tensor((width, height, width, height), device=device)
    
    with dt[1]:
        preds, train_out = model(im)
    preds = preds.detach()

    id = int(model_name.split('/')[-1].split('.')[0].split('_')[-1])
    # Loss
    if id < 5:
        loss += compute_loss1(train_out, teacher_labels)[1]  # box, obj, cls
    else:
        loss += compute_loss2(train_out, teacher_labels)[1]  # box, obj, cls

    # NMS
    teacher_labels[:, 2:] *= torch.tensor((width, height, width, height), device=device)  # to pixels
    lb = [] # for autolabelling

    with dt[2]:
        preds = non_max_suppression(
            preds, conf_thres, iou_thres, labels=lb, multi_label=True, agnostic=single_cls, max_det=max_det
        )

    # Metrics
    for si, pred in enumerate(preds):
        labels = teacher_labels[teacher_labels[:, 0] == si, 1:]
        nl, npr = labels.shape[0], pred.shape[0]  # number of labels, predictions
        path, shape = Path(paths[si]), shapes[si][0]
        correct = torch.zeros(npr, niou, dtype=torch.bool, device=device)  # init
        seen += 1

        if npr == 0:
            if nl:
                stats.append((correct, *torch.zeros((2, 0), device=device), labels[:, 0]))
                if plots:
                    confusion_matrix.process_batch(detections=None, labels=labels[:, 0])
            continue

        # Predictions
        if single_cls:
            pred[:, 5] = 0
        predn = pred.clone()
        scale_boxes(im[si].shape[1:], predn[:, :4], shape, shapes[si][1])  # native-space pred

        # Evaluate
        if nl:
            tbox = xywh2xyxy(labels[:, 1:5])  # target boxes
            scale_boxes(im[si].shape[1:], tbox, shape, shapes[si][1])  # native-space labels
            labelsn = torch.cat((labels[:, 0:1], tbox), 1)  # native-space labels
            correct = process_batch(predn, labelsn, iouv)
            if plots:
                confusion_matrix.process_batch(predn, labelsn)
        stats.append((correct, pred[:, 4], pred[:, 5], labels[:, 0]))  # (correct, conf, pcls, tcls)


    # Plot images
    if plots and batch_i < 3:
        plot_images(im, teacher_labels, paths, save_dir / f"val_batch{batch_i}_labels.jpg", names)  # labels
        plot_images(im, output_to_target(preds), paths, save_dir / f"val_batch{batch_i}_pred.jpg", names)  # pred
        
    model.cpu()

# Compute metrics
stats = [torch.cat(x, 0).cpu().numpy() for x in zip(*stats)]  # to numpy
if len(stats) and stats[0].any():
    tp, fp, p, r, f1, ap, ap_class = ap_per_class(*stats, plot=plots, save_dir=save_dir, names=names)
    ap50, ap = ap[:, 0], ap.mean(1)  # AP@0.5, AP@0.5:0.95
    mp, mr, map50, map = p.mean(), r.mean(), ap50.mean(), ap.mean()
nt = np.bincount(stats[3].astype(int), minlength=nc)  # number of targets per class

# Print results
pf = "%22s" + "%11i" * 2 + "%11.3g" * 4  # print format
LOGGER.info(pf % ("all", seen, nt.sum(), mp, mr, map50, map))
if nt.sum() == 0:
    LOGGER.warning(f"WARNING ⚠️ no labels found in {task} set, can not compute metrics without labels")

# Print results per class
if (verbose or (nc < 50 and not training)) and nc > 1 and len(stats):
    for i, c in enumerate(ap_class):
        LOGGER.info(pf % (names[c], seen, nt[c], p[i], r[i], ap50[i], ap[i]))

# Print speeds
t = tuple(x.t / seen * 1e3 for x in dt)  # speeds per image
if not training:
    shape = (batch_size, 3, imgsz, imgsz)
    LOGGER.info(f"Speed: %.1fms pre-process, %.1fms inference, %.1fms NMS per image at shape {shape}" % t)

# Plots
if plots:
    confusion_matrix.plot(save_dir=save_dir, names=list(names.values()))

# Return results
if not training:
    s = ""
    LOGGER.info(f"Results saved to {colorstr('bold', save_dir)}{s}")
maps = np.zeros(nc) + map
for i, c in enumerate(ap_class):
    maps[c] = ap[i]


      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
 11%|█         | 10/93 [00:17<02:21,  1.70s/it]
                   all        320       6239      0.512      0.164      0.172      0.105
Speed: 0.1ms pre-process, 10.7ms inference, 0.8ms NMS per image at shape (32, 3, 640, 640)
Results saved to exp_2


In [14]:
torch.cuda.empty_cache()

In [15]:
print(len(stats[3]))

6239


In [16]:
print((mp, mr, map50, map, *(loss.cpu() / len(train_loader)).tolist()), maps, t)

(0.5118911432068413, 0.1642187224292031, 0.17185789158503412, 0.10483564129379287, 0.005577079486101866, 0.024473417550325394, 0.003959154710173607) [    0.22006     0.11626     0.31403     0.14741     0.10484      0.1358     0.24865     0.13903     0.10484    0.098566           0   0.0099984           0      0.1089     0.10484     0.10484           0     0.10484     0.10484     0.10484     0.10484     0.10484     0.10484     0.10484    0.027928    0.077328
    0.068934      0.1722     0.12057     0.10484     0.10484     0.10484           0     0.10484     0.10484     0.10484           0     0.10484           0     0.10484     0.10484     0.10484     0.10484     0.10484     0.10484     0.10484           0     0.10484     0.10484     0.10484     0.10484     0.10484
     0.10484     0.10484     0.10484     0.10484     0.34566     0.10484    0.015462     0.10484      0.5686     0.10484     0.10484     0.10484     0.10484     0.10484           0     0.10484     0.10484     0.10484     0.10